In [6]:
from landlab import RasterModelGrid
from landlab.components import FlowAccumulator, DepressionFinderAndRouter
import matplotlib.pyplot as plt
import numpy as np
import lsdnumbatools as lsdnb
import numpy as np
import numba as nb
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
import cython
import numpy as np
import CHONK_II.cythonk.cychonk as cychonk
%matplotlib widget

ny,nx = 500,500
mg = RasterModelGrid((ny, nx), xy_spacing=30)
zeta = np.copy(mg.add_field("topographic__elevation", mg.node_x.copy(), at="node"))
zeta += 0.01 * mg.node_y
zeta = zeta.reshape(ny,nx)

def landlab_acc(elev):
    mg = RasterModelGrid((ny, nx), xy_spacing=30)
    z = mg.add_field("topographic__elevation", elev.ravel(), at="node")
    fr = FlowAccumulator(mg, flow_director='FlowDirectorD8', depression_finder = 'DepressionFinderAndRouter')
    fr.run_one_step() 
    df = DepressionFinderAndRouter(mg, reroute_flow = True)
    df.map_depressions()  # reroute_flow defaults to True
    fr.run_one_step() 
    
def lsdnbtools(elev):
    putna = lsdnb.DEM()
    putna.set_main_raster(elev, nx, ny, 30, 30, node_type = None, xmin = None, xmax = None, ymin = None, ymax = None, ymin_bottom = True)
    putna.compute_graph()
    DA = putna.graph.accumulate_down_constant_D8S(900,return_2D = True, dtype = np.float64)

def chonk_ii(elev):
    tg = cychonk.pygraph(b"SF", b"D8")
    # print("1")
    tg.set_dimensions(ny,nx, ny*nx, 30, 30)
    # print("2")
    tg.set_default_boundaries(b"4edges")
    tg.compute_graph(b"both", elev.ravel().astype(np.float64))
    DA = tg.accumulate_downstream(900)

In [7]:
print(zeta.shape)
lsdnbtools(zeta)
landlab_acc(zeta)
chonk_ii(zeta)

(500, 500)


In [8]:
print('landlab')
%timeit landlab_acc(zeta)
print('lsdnumbatools (similar algorithms to CHONK)')
%timeit lsdnbtools(zeta)
print('CHONK_v2 (no depression handling yet!!!)')
%timeit lsdnbtools(zeta)

landlab
1.78 s ± 46.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
lsdnumbatools (similar algorithms to CHONK)
68.1 ms ± 2.84 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
CHONK_v2 (no depression handling yet!!!)
65.9 ms ± 1.94 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
fr = FlowAccumulator(mg, flow_director='D8')

In [34]:
fr.run_one_step()  # the flow "gets stuck" in the hole

In [20]:
fig,ax = plt.subplots()

ax.imshow(np.log10(mg.at_node['drainage_area'].reshape(mg.shape)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

FieldError: drainage_area

In [37]:
fr = FlowAccumulator(mg, flow_director='FlowDirectorD8', depression_finder = 'DepressionFinderAndRouter')
df = DepressionFinderAndRouter(mg, reroute_flow = True)
df.map_depressions()  # reroute_flow defaults to True
fr.run_one_step()  

In [38]:
fig,ax = plt.subplots()

ax.imshow(np.log10(mg.at_node['drainage_area'].reshape(mg.shape)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/var/folders/t0/7zf23f454vn3_kkgrbf4c1500000gp/T/ipykernel_17186/3777550170.py:3: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(mg.at_node['drainage_area'].reshape(mg.shape)))


In [29]:
?FlowAccumulator

Init signature: FlowAccumulator(*args, **kwds)
Docstring:     
Component to accumulate flow and calculate drainage area.

This is accomplished by first finding flow directions by a user-specified
method and then calculating the drainage area and discharge.

Optionally, spatially variable runoff can be set either by the model grid
field 'water__unit_flux_in' or the input variable *runoff_rate**.

Optionally a depression finding component can be specified and flow
directing, depression finding, and flow routing can all be accomplished
together.

NOTE: The perimeter nodes  NEVER contribute to the accumulating flux, even
if the  gradients from them point inwards to the main body of the grid.
This is because under Landlab definitions, perimeter nodes lack cells, so
cannot accumulate any discharge.

FlowAccumulator stores as ModelGrid fields:

    -  Node array of drainage areas: *'drainage_area'*
    -  Node array of discharges: *'surface_water__discharge'*
    -  Node array containing down